<a href="https://colab.research.google.com/github/Sergey26/import-export-rus/blob/main/import_export.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title
from google.colab import files

uploaded = files.upload()

for filename in uploaded.keys():
  print(f'Загружен файл "{filename}" длиной {len(uploaded[filename])} байт')


# @title
import os
import zipfile
import pandas as pd

# Извлечение всех ZIP-файлов
def extract_zip_files():
    zip_contents = {}
    for zip_file in os.listdir('.'):
        if zip_file.endswith('.zip'):
            with zipfile.ZipFile(zip_file, 'r') as zip_ref:
                zip_ref.extractall('.')
                for file_name in zip_ref.namelist():
                    zip_contents[file_name] = zip_file
                print(f'Файл {zip_file} разархивирован')
    return zip_contents

zip_contents = extract_zip_files()

# Обработка Excel файлов
def process_excel_files(zip_contents):
    results = []
    for file, zip_file in zip_contents.items():
        if file.endswith('.xlsx') or file.endswith('.xls'):
            try:
                df = pd.read_excel(file, header=1, sheet_name="Декларации")
                # Идентификация столбцов
                country_col = next((col for col in df.columns if "Страна отправления" in col), None)
                name_col_sender = next((col for col in df.columns if "Наименование.ФИО отправителя" in col), None)
                name_col_receiver = next((col for col in df.columns if "Наименование.ФИО получателя" in col), None)

                if country_col and (name_col_sender or name_col_receiver):
                    # Определение активного столбца для работы
                    active_name_col = name_col_sender if name_col_sender else name_col_receiver

                    # Получение уникальных стран и соответствующих записей
                    unique_countries = df[country_col].unique()
                    sender_info = []
                    for country in unique_countries:
                        country_df = df[df[country_col] == country]
                        senders = country_df[active_name_col].dropna().unique()
                        if len(unique_countries) < 3:
                            selected_senders = senders[:4]  # Берем до 4 записей, если стран меньше 3
                        else:
                            selected_senders = senders[:1]  # Иначе берем только первую запись
                        sender_info.append((country, list(selected_senders)))
                else:
                    sender_info = "Не найдены столбцы со страной отправления или информацией об отправителе/получателе"

                results.append((zip_file, file, sender_info))
            except Exception as e:
                print(f'Ошибка при обработке файла {file}, из архива {zip_file}: {e}')
    return results

# Вывод результатов
results = process_excel_files(zip_contents)
for zip_file, file, sender_info in results:
    print(f'Исходный ZIP: {zip_file}, Файл: {file}, Информация: {sender_info}')

